# Filings Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import json
import gspread
import re
from pytz import timezone
from config import Config
from pathlib import Path
from jinja2 import Template

from datetime import datetime, timedelta, tzinfo, timezone
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

Define the scope of the report for corp types and filing types

In [2]:
corpTypesStr = ''
corpTypeList = ['BC', 'BEN', 'C', 'CBEN', 'CC', 'CCC', 'CUL', 'ULC']
for x in range(len(corpTypeList)):
    corpTypesStr += '\'' + corpTypeList[x] + '\'' + ','
corpTypesStr = '('+corpTypesStr[:-1]+')'   

filingTypesDic = {'alteration': ['CHANGES OF NAME', 'CHANGES OF TYPE']
                  , 'incorporationApplication': ['INCORPORATIONS']
                #   , 'amalgamationApplication': ['AMALGAMATIONS']
                #   , 'continuationIn': ['CONTINUATIONS IN']
                  , 'continuationOut': ['CONTINUATIONS OUT']
                  , 'dissolution': ['DISSOLUTIONS']
                  , 'restoration': ['RESTORATIONS']
                  , 'registration': ['REGISTRATIONS']}

filingTypesStr = '' 
for i in filingTypesDic:
    filingTypesStr += '\'' +i + '\'' + ','
filingTypesStr = '('+filingTypesStr[:-1]+')'     

# Following dictionary defined the value of the corp type which is not exact the same as the full desc in the corp_type table 
# so it will display on the report as required.
corp_type_dict = {'BC': 'BC Company', 'BEN': 'BC Benefit Company', 
                    'C': 'Continue in BC Limited Company', 'CBEN': 'Continue in BC Benefit Company',
                    'CC': 'BC Community Contribution Company', 'CCC': 'BC Community Contribution Company', 
                    'CUL': 'Continuation in BC Unlimited Liability Company', 'ULC': 'BC Unlimited Liability Company'}

corp_class_dict = {'BC': 'BC Company(s)', 'XPRO': 'Extraprovincial Company(s)', 'SOC': 'BC Societies', 'FIRM': 'BC Partnership'}

"('alteration','incorporationApplication','continuationOut','dissolution','restoration','registration')"

In [ ]:
def generateOutputList(df, v_category, items_list, unsupport_items_list):
    if not df.empty:        
        t_category_values = []
        t_sub_category_values = []
        t_corp_class_values = []
        t_desc_values = [] 
        seq = ''
                
        for ind, row in df.iterrows(): 
            seq = df['legal_name'] + ' at ' +  df['effective_date'].dt.strftime('%I:%M %p') + ' Pacific Time' 
            if v_category == 'CHANGES OF NAME':               
                t_category_values.append(v_category)  
                t_sub_category_values.append('')   
                if corp_class_dict.get(row['corp_class']) != None:  
                    t_corp_class_values.append(corp_class_dict.get(row['corp_class']))                                        
                    t_desc_values.append('The Registrar of Companies hereby gives notice that the following ' + corp_class_dict.get(row['corp_class']).replace('y(s)', 'ies').replace('BC ', '').lower() + ' have changed their names')                
                    seq = df['from_name'] + ' to ' +  df['to_name'] + ' at ' +  df['effective_date'].dt.strftime('%I:%M %p') + ' Pacific Time' 
                else:
                    t_corp_class_values.append('') 
                    t_desc_values.append('UNSUPPORT')    
            elif v_category == 'CHANGES OF TYPE':               
                if  (corp_type_dict.get(row['from_type']) != None and corp_type_dict.get(row['to_type']) != None):
                    t_category_values.append('ALTERATION TO BECOME A ' + corp_type_dict.get(row['to_type']).upper())        
                    t_sub_category_values.append('') 
                    t_corp_class_values.append(corp_class_dict.get(row['corp_class']))           
                    t_desc_values.append('The Registrar of Companies hereby gives notice that the following ' + corp_type_dict.get(row['from_type'])[0:-1] + 'ies filed a notice of alteration to become ' + corp_type_dict.get(row['to_type'])[0:-1] + 'ies')                  
                    seq = corp_type_dict.get(row['from_type']) + ' became ' +  corp_type_dict.get(row['to_type']) + ' at ' +  df['effective_date'].dt.strftime('%I:%M %p') + ' Pacific Time'  
                else:
                    t_category_values.append('')
                    t_sub_category_values.append('')
                    t_corp_class_values.append('')   
                    t_desc_values.append('UNSUPPORT')            
            #  CONTINUATIONS IN has not been implemented yet. seq value need to be updated after it is in the filing table.                         
            # elif v_category == 'CONTINUATIONS IN':  
            #         t_category_values.append(v_category)
            #         t_sub_category_values.append('')                     
            #         t_corp_class_values.append(corp_class_dict.get(row['corp_class']))  
            #         t_desc_values.append('The Registrar of Companies hereby gives notice that the following foreign corporations have continued into British Columbia')   # checked
            elif v_category == 'CONTINUATIONS OUT':              
                    t_category_values.append(v_category)
                    t_sub_category_values.append('') 
                    t_corp_class_values.append(corp_class_dict.get(row['corp_class']))  
                    t_desc_values.append('The Registrar of Companies hereby gives notice that the following companies were removed from the registry and continued out of British Columbia')   # checked                    
                    seq = df['legal_name'] + df['foreign_text'] 
                    # don't want time... + ' at ' +  df['effective_date'].dt.strftime('%I:%M %p') + ' Pacific Time'             
            else: 
                t_category_values.append(v_category)   
                if v_category == 'INCORPORATIONS':
                    t_sub_category_values.append('') 
                    if corp_class_dict.get(row['corp_class']) != None:
                            t_corp_class_values.append(corp_class_dict.get(row['corp_class']))  
                    else: 
                        t_corp_class_values.append('')  
                    t_desc_values.append('The Registrar of Companies hereby gives notice of the incorporation of the following companies')                               
                # AMALGAMATIONS has not been implement yet. seq value need to be updated after it is in the filing table.     
                elif v_category == 'AMALGAMATIONS':   
                    t_sub_category_values.append('') 
                    if row['corp_class'] == 'BC': 
                        t_corp_class_values.append(corp_class_dict.get(row['corp_class']))  
                        t_desc_values.append('The Registrar of Companies hereby gives notice that the following companies have amalgamated')                                          
                    elif row['corp_class'] == 'XPRO' and (row['corp_type_cd'] == 'A' or row['corp_type_cd'] == 'B'): 
                        t_corp_class_values.append('BC Company(s) with Foreign Corporations')
                        t_desc_values.append('The Registrar of Companies hereby gives notice that the following companies and foreign corporations have amalgamated')                                               
                    elif row['corp_class'] == 'XPRO' and row['corp_type_cd'] == 'FOR':
                        t_corp_class_values.append('Foreign Entities')
                        t_desc_values.append('The Registrar of Companies hereby gives notice that the following extraprovincial companies have registered an amalgamation')                                                 
                    else:
                        t_corp_class_values.append('') 
                        t_desc_values.append('UNSUPPORT')   
                elif v_category == 'DISSOLUTIONS':
                    if row['filing_sub_type'] != None:
                        t_sub_category_values.append(row['filing_sub_type'].capitalize()) 
                        if corp_class_dict.get(row['corp_class']) != None:
                            t_corp_class_values.append(corp_class_dict.get(row['corp_class']))  
                        else: 
                            t_corp_class_values.append('')                           
                        t_desc_values.append('The Registrar of Companies hereby gives notice that the following companies were dissolved as a result of a ' + row['filing_sub_type'] + ' dissolution')  
                    else:
                        t_sub_category_values.append('') 
                        if corp_class_dict.get(row['corp_class']) != None:
                            t_corp_class_values.append(corp_class_dict.get(row['corp_class']))  
                        else: 
                            t_corp_class_values.append('') 
                        t_desc_values.append('The Registrar of Companies hereby gives notice that the following companies were dissolved under section 317, 422 or 423 of the Business Corporations Act')    
                elif v_category == 'REGISTRATIONS':
                    t_sub_category_values.append('') 
                    if corp_class_dict.get(row['corp_class']) != None:
                        t_corp_class_values.append(corp_class_dict.get(row['corp_class']))  
                        if row['corp_class'] == 'XPRO':
                            t_desc_values.append('The Registrar of Companies hereby gives notice of the registration of the following foreign entities as extraprovincial companies')   
                        else:
                            t_desc_values.append('The Registrar of Companies hereby gives notice of the registration of the following companies')     
                    else: 
                        t_corp_class_values.append('')  
                        t_desc_values.append('UNSUPPORT') 
                elif v_category == 'RESTORATIONS':    #it is not in filings table yet. seq value need to be updated after it is in the filing table.       
                    t_sub_category_values.append('')      
                    if row['filing_sub_type'] == 'fullRestoration' or  \
                       row['filing_sub_type'] == 'limitedRestoration' or \
                       row['filing_sub_type'] == 'limitedRestorationExtension' or \
                       row['filing_sub_type'] == 'expiredLimitedRestoration' or \
                       row['filing_sub_type'] == 'limitedRestorationToFull': 
                        str = re.sub(r"\B([A-Z])", r" \1", row['filing_sub_type'])  
                        str = str[:1].upper() + str[1:]                         
                        if corp_class_dict.get(row['corp_class']) != None:
                            t_corp_class_values.append(corp_class_dict.get(row['corp_class']))  
                            if row['corp_class'] == 'XPRO':
                                t_desc_values.append('The Registrar of Companies hereby gives notice of the ' + str.lower() + ' of the following foreign entities as extraprovincial companies') 
                            else:    
                                t_desc_values.append('The Registrar of Companies hereby gives notice of the ' + str.lower() + ' of the following companies') 
                        else: 
                            t_corp_class_values.append('') 
                            t_desc_values.append('UNSUPPORT') 
                    else:
                        t_corp_class_values.append('') 
                        t_desc_values.append('UNSUPPORT')                              
        
      
        # put these fields in the front of the df        
        df = pd.concat([pd.Series(seq, index=df.index, name='text'), df], axis=1)  # fifth column
        df = pd.concat([pd.Series(t_desc_values, index=df.index, name='t_desc'), df], axis=1)  # fourth column
        df = pd.concat([pd.Series(t_corp_class_values, index=df.index, name='t_corp_class'), df], axis=1)  # third column
        df = pd.concat([pd.Series(t_sub_category_values, index=df.index, name='t_sub_category'), df], axis=1)  # second column        
        df = pd.concat([pd.Series(t_category_values, index=df.index, name='t_category'), df], axis=1)   # first column

        for i, x in df.groupby(['t_desc']):  
            for k, gr in df.loc[df['t_desc'] == i].groupby([df['effective_date'].dt.strftime('%Y %m, %d')]):
                if 'UNSUPPORT' in gr.values:
                    unsupport_items_list.append(list(gr.values.tolist()))
                else:                                    
                    items_list.append(list(gr.values.tolist()))            

In [ ]:
%%sql 
INSERT INTO sent_to_gazette (filing_id, filing_type, identifier, sent_to_gazette_date)
SELECT f.id, f.filing_type, b.identifier, null
FROM businesses b, filings f
WHERE b.id = f.business_id
    AND b.legal_type in {corpTypesStr} 
    AND f.status='COMPLETED'    
    AND f.source='LEAR'
    AND f.filing_type in {filingTypesStr} 
    AND f.id not in (select filing_id from sent_to_gazette)
    AND to_char(f.filing_date at time zone 'America/Vancouver','yyyymmdd') >= to_char(now() at time zone 'America/Vancouver' - interval '9 day','yyyymmdd')   
    AND to_char(f.filing_date at time zone 'America/Vancouver','yyyymmdd') <= to_char(now() at time zone 'America/Vancouver' - interval '3 day','yyyymmdd');

In [ ]:
%%sql  all_data  <<
    SELECT b.identifier, b.legal_name, b.legal_type, f.effective_date, f.filing_type, f.id, f.status, ct.corp_class, ct.full_desc, f.filing_sub_type,
    f.meta_data-> 'alteration'->>'fromLegalType' as from_type, 
    f.meta_data-> 'alteration'->>'toLegalType' as to_type, 
    f.meta_data-> 'alteration'->>'fromLegalName' as from_name, 
    f.meta_data-> 'alteration'->>'toLegalName' as to_name,
    ' (continued into '||coalesce(foreign_jurisdiction,' ')||case when foreign_jurisdiction_region is null then ' ' else '-'
    ||foreign_jurisdiction_region end||'), being continued under the name '||coalesce(foreign_legal_name,' ') AS foreign_text
    FROM businesses b, corp_types ct, filings f, sent_to_gazette stg
    WHERE b.id = f.business_id
    AND f.id=stg.filing_id
    AND b.legal_type = ct.corp_type_cd
    AND stg.sent_to_gazette_date is null
    ORDER BY 9, 8, 4;

In [ ]:
all_df = all_data.DataFrame()

items_list = []
unsupport_items_list = []
sent_to_gazette_filename = os.path.join(os.getcwd(), r'data/')+'sent_to_gazette.xml'
if not os.path.isfile(sent_to_gazette_filename):
        Path(sent_to_gazette_filename).touch()

if not all_df.empty:
    all_df['effective_date'] = all_df['effective_date'].dt.tz_convert('US/Pacific')
    for key in filingTypesDic:
        df = all_df.loc[all_df['filing_type'] == key]
        if (key == 'alteration'): 
            generateOutputList(df.loc[df['to_name'].ne(df['from_name'])] , filingTypesDic[key][0], items_list, unsupport_items_list)  # for 'CHANGES OF NAME'
            for x in range(len(corpTypeList)): 
                generateOutputList(df.loc[df['to_type'].ne(df['from_type'])].loc[df['to_type'] == corpTypeList[x]], filingTypesDic[key][1], items_list, unsupport_items_list)  # for 'CHANGES OF TYPE'
        else:
            generateOutputList(df, filingTypesDic[key][0], items_list, unsupport_items_list)     

    headList = [] 
    for i in items_list:
        # To make sure there is no duplicated on the header
        if (str(i[0][0]) + ':' + str(i[0][1]) + ':' + str(i[0][2]) + ':' + str(i[0][3])) not in headList:
            headList.append(str(i[0][0]) + ':' + str(i[0][1]) + ':' + str(i[0][2]) + ':' + str(i[0][3])) 

    data = { 
            'headItems': list(headList),
            'filingItems': list(items_list)        
            }

    template = Path(f'{Config.TEMPLATE_PATH}/public_notice.xml').read_text()    
    jnja_template = Template(template, autoescape=True)
    output_from_parsed_template = jnja_template.render(data)

with open(sent_to_gazette_filename, 'w') as fh:
    if all_df.empty:
        fh.write('No data available during this period')
    else:
        fh.write(output_from_parsed_template)  
        
       

In [ ]:
df = pd.DataFrame(unsupport_items_list)    

upstr = f'999999,'
for idx, row in df.iterrows():
    for col in list(df.columns):
        if df[col][idx] !=None:
            upstr += f'''{df[col][idx][10]},'''

upstr = upstr[:-1]+')'   


In [ ]:
%%sql  all  << 
UPDATE sent_to_gazette set sent_to_gazette_date=now(), status='unsupport' where filing_id in ({upstr[:-1]})